In [244]:
# Importing necessary libraries, loading in locally available data
import json
import pandas as pd
import numpy as np
from urllib import request 
from bs4 import BeautifulSoup
import re
import plotly.express as px
import us

# Importing latest data on COVID cases at state, county level
csvfile = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'

# the csv file reader returns a list of the csv items on each line
COVReader = pd.read_csv(csvfile,  dialect='excel', delimiter=',', parse_dates = ['date'])
covid_data = pd.DataFrame(COVReader)
covid_data = covid_data.set_index('state')
covid_data


,date,county,fips,cases,deaths
state,,,,,
Washington,2020-01-21,Snohomish,53061.0,1,0
Washington,2020-01-22,Snohomish,53061.0,1,0
Washington,2020-01-23,Snohomish,53061.0,1,0
Illinois,2020-01-24,Cook,17031.0,1,0
Washington,2020-01-24,Snohomish,53061.0,1,0
...,...,...,...,...,...
Wyoming,2020-05-29,Sweetwater,56037.0,26,0
Wyoming,2020-05-29,Teton,56039.0,100,1
Wyoming,2020-05-29,Uinta,56041.0,12,0


In [245]:
# Scrape the Web for HTML code used to build embedded visualizations on the AEI site.
AEI_URL = "https://e.infogram.com/_/bo5pjUi7dprQAvs1l6oZ?src=embed"
html = request.urlopen(AEI_URL).read().decode('utf8') 
# Convert the raw HTML into a beautiful soup object
htmlsoup = BeautifulSoup(html,"html.parser")
scripts = htmlsoup.find_all('script')
# Use the 're' library to extract the JSON script embedded in the HTML code of the infographic on site
json_string=re.search("window.infographicData=({.*})",scripts[4].string).group(1)
graphic_json = json.loads(json_string)
# Pull only the JSON elements that build the visualization
gj_chart = graphic_json.get('elements', {}).get('content'
                                                , {}).get('content', {}).get('entities'
                                                                             , {}).get('8bbc0f22-218e-4ad9-aa1d-262e2e6aa154', {}).get('props')
gj_data = gj_chart.get('chartData', {}).get('data')
# Take only the first list
data = gj_data[0]
# Take this list and transform into a pandas dataframe
df = pd.DataFrame(data)
# Drop all columns with no values
df = df.dropna(axis=1, how='any')
# Create list of column names based on contents
#Take first row of data frame, Extract text before colon as column names
column_names = []
for d in df.iloc[1]:
    col = []
    col.append(re.findall("(.*):",d))
    for c in col:
        column_names.append(c)
new_df = pd.DataFrame(column_names)
names = new_df.sum(axis=1).tolist()
df.columns = names
#Use same function in an inplace form to get cleaned up yes/no
def clean_series(cell):
    if type(cell) is str:
        cell_list = cell.split(':')
        if len(cell_list) > 1:
            cell = cell_list[1].strip()
    return cell
df = df.applymap(clean_series)
df.columns = ['State','b1','b2','latlon','state_blank','state_emp_travel','school_closure','national_guard','ne_business_closed','bar_res_closed','curfew','gather_limit','hospital_cap_inc','relax_licensure','mand_vis_quar','postponed_primary','stay_at_home','elective_surgery_post','elective_surg']
df = df.drop(['b1','b2','state_blank','elective_surg','latlon'], axis = 1)
df = df.set_index('State')
df

,state_emp_travel,school_closure,national_guard,ne_business_closed,bar_res_closed,curfew,gather_limit,hospital_cap_inc,relax_licensure,mand_vis_quar,postponed_primary,stay_at_home,elective_surgery_post
State,,,,,,,,,,,,,
Alabama,No,Yes,Yes,Yes,Yes,No,25,No,No,No,Yes,Yes,No
Alaska,Yes,Yes,Yes,Yes,Yes,No,10,No,No,Yes,No,Yes,Yes
Arizona,No,Yes,Yes,Yes,Yes,No,No limit,Yes,No,Yes,No,Yes,No
Arkansas,Yes,Yes,Yes,No,Yes,No,No limit,No,No,No,No,No,No
California,No,Yes,Yes,No,Yes,No,<10,No,No,No,No,Yes,No
Colorado,No,Yes,Yes,Yes,Yes,No,10,No,Yes,No,No,Yes,Yes
Connecticut,Yes,Yes,Yes,Yes,Yes,Recommended,50,No,No,No,Yes,Yes,No
Delaware,No,Yes,Yes,Yes,Yes,No,50,No,No,Yes,Yes,Yes,No
Florida,Yes,Yes,Yes,Yes,Yes,No,No limit,No,No,Yes,No,Yes,Yes


In [246]:
# Answer Question 1
# Calculate Intervention Score on State Data
yes_nos_to_convert = ['state_emp_travel','school_closure','national_guard','ne_business_closed','bar_res_closed','curfew','hospital_cap_inc','relax_licensure','mand_vis_quar','postponed_primary','stay_at_home','elective_surgery_post']
df = df.replace(to_replace = ['Yes','No','Recommended'],value = [1,0,1])
# Create a map based on this data
# Set dataframe with intervention
map_df = pd.DataFrame(df.sum(axis=1))
map_df.reset_index(level=map_df.index.names, inplace=True)
map_df = map_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# Set up lookup dataframe
state_lookup = pd.DataFrame.from_dict(us.states.mapping('name', 'abbr'),orient='index',columns=['abbr'])
state_lookup.index.names = ['State']
state_lookup.reset_index(level=state_lookup.index.names, inplace=True)
state_lookup = state_lookup.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# Join the two
map_df = map_df.merge(state_lookup, left_on='State', right_on='State')
map_df.columns=["state","intervention_score",'abbr']
#test_df = pd.concat([map_df,state_lookup])
fig = px.choropleth(data_frame = map_df,locations='abbr', locationmode="USA-states", color='intervention_score',color_continuous_scale="Viridis", scope="usa")
fig.show()

In [248]:
# Answer Question 2
q2_df = df.drop(['gather_limit'], axis = 1)
# Create new data frame to store grouped results
output = pd.DataFrame()
output['%_of_states'] = q2_df.sum()
output['%_of_states'] = round(output['%_of_states']/51*100,1)
output = output.sort_values(by='%_of_states', ascending=False)
fig = px.bar(output, y='%_of_states',color='%_of_states',color_continuous_scale=px.colors.sequential.Cividis_r)
fig.show()

In [251]:
# Answer Question 3

# Create calculation for new cases, new deaths, for each state over time
grouped_state = covid_data.groupby(['state','date']).sum().drop('fips',axis=1)
grouped_state.reset_index(level=grouped_state.index.names, inplace=True)
#grouped_state.reset_index(level=1, inplace=True)
# Create new grouped data frame of state, intervention level, new cases, new deaths
# Join both imported datasets on state name
df_joined = grouped_state.merge(map_df, left_on='state', right_on='state')
df_joined.drop(['state','abbr'] ,axis=1,inplace=True)
# Calculate intervention Level
df_joined["intervention_level"] = np.where(df_joined["intervention_score"]<=4, "Low Intervention", np.where((df_joined["intervention_score"] >= 5) & (df_joined["intervention_score"] <= 8), "Medium Intervention", "High Intervention"))
df_joined.head(100)
# Calculate new case rate, new death rate
grouped_intervention = df_joined.groupby(['intervention_level','date']).sum().drop('intervention_score',axis=1)
# Create final dataframe for chart
output = pd.DataFrame()
output['cases'] = grouped_intervention.cases
output['deaths'] = grouped_intervention.deaths
output['new_case_rate']=output.cases.pct_change() * 100
output['new_death_rate']=output.deaths.pct_change() * 100
output.reset_index(level=output.index.names, inplace=True)
# Plot new rates over time
fig = px.line(output, x="date", y="new_case_rate", color = "intervention_level", title='New COVID Case Rate over time, by intervention level')
fig.show()
fig2 = px.line(output, x="date", y="new_death_rate", color = "intervention_level", title='New COVID Death Rate over time, by intervention level')
fig2.show()